In [ ]:
# --- 1. ランタイムをリセットした後、このセルから実行 ---

# 既存の関連パッケージを徹底的にアンインストール
# 競合の原因となる可能性のあるライブラリを全て削除します
!pip uninstall -y \
    tensorflow \
    tf-keras \
    keras \
    protobuf \
    jax \
    jaxlib \
    mediapipe-model-maker \
    grpcio-status \
    orbax-checkpoint \
    dopamine-rl \
    flax \
    ydf \
    tensorflow-decision-forests \
    thinc \
    seqeval \
    inflect \
    typeguard # typeguardもinflectとの競合回避のため

# MediaPipe Model Makerが要求する厳密なバージョンのパッケージをインストール
# --force-reinstall を付けて、既存のバージョンがあっても上書きします。
# --no-deps を使うと依存関係を無視しますが、今回は依存関係解決が目的なので使いません。
print("Installing core dependencies...")
!pip install \
    mediapipe-model-maker==0.2.1.4 \
    tensorflow==2.15.1 \
    tf-keras==2.15.1 \
    protobuf==3.20.3 \
    jax==0.4.34 \
    jaxlib==0.4.34 \
    --quiet \
    --force-reinstall

# 正常にインストールされたか確認
print("\n--- Installed versions check ---")
!pip show mediapipe-model-maker tensorflow tf-keras protobuf jax jaxlib

# まだ競合メッセージが出る可能性はありますが、
# 必要なライブラリが指定バージョンでインストールされているか確認してください。
print("\n--- Final dependency resolution check ---")
!pip check

Installing core dependencies...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
spacy 3.8.7 requires thinc<8.4.0,>=8.3.4, which is not installed.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
pylibcudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 20.0.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.1 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you h

In [ ]:
# インポート
import os
import tensorflow as tf
from mediapipe_model_maker import object_detector
from mediapipe_model_maker import quantization
import zipfile
import shutil
from google.colab import files

# TensorFlowのバージョンを確認
assert tf.__version__.startswith('2'), "TensorFlow 2.xがインストールされていることを確認してください。"

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import xml.etree.ElementTree as ET # XMLをパースするためのライブラリ

# --- 変換関数の定義 ---
def convert_to_pascal_voc_structure(base_path):
    """
    指定されたパス直下の .jpg と .xml ファイルを、
    PASCAL VOC形式のサブディレクトリ (images, Annotations) に移動し、
    XMLファイル内の <path> タグを削除する。
    """
    print(f"'{base_path}' の構造を変換しています...")
    images_dir = os.path.join(base_path, "images") # 今回のエラーメッセージに合わせて 'images' に固定
    annotations_dir = os.path.join(base_path, "Annotations")

    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(annotations_dir, exist_ok=True)

    for filename in os.listdir(base_path):
        filepath = os.path.join(base_path, filename)
        if os.path.isfile(filepath):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                shutil.move(filepath, os.path.join(images_dir, filename))
            elif filename.lower().endswith('.xml'):
                # XMLファイルを読み込み、<path> タグを削除
                tree = ET.parse(filepath)
                root = tree.getroot()
                path_element = root.find('path')
                if path_element is not None:
                    root.remove(path_element)
                    # 修正したXMLを同じファイル名で上書き保存
                    tree.write(filepath)
                    print(f"  XMLファイル '{filename}' から <path> タグを削除しました。")
                shutil.move(filepath, os.path.join(annotations_dir, filename))
    print(f"'{base_path}' の変換が完了しました。")

In [ ]:
# --- 1. データセットのアップロードと解凍 ---
print("データセット(zipファイル)をアップロードしてください (PASCAL VOC形式, XML + JPEG)...")
try:
    uploaded = files.upload()

    # アップロードされたZIPファイルを特定し、解凍します。
    zip_file_name = None
    for fn in uploaded.keys():
        if fn.endswith('.zip'):
            zip_file_name = fn
            break

    if zip_file_name:
        print(f"'{zip_file_name}' を解凍しています...")
        extract_dir = "pascal_voc_dataset_extracted"
        os.makedirs(extract_dir, exist_ok=True) # 解凍先ディレクトリを作成
        with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print("解凍が完了しました。")

        # ZIPファイル直下に 'train' と 'test' ディレクトリがあることを想定
        train_dataset_path_raw = os.path.join(extract_dir, "train")
        validation_dataset_path_raw = os.path.join(extract_dir, "test")

        # パスが存在するか確認
        if not os.path.exists(train_dataset_path_raw):
            raise FileNotFoundError(f"訓練データセットのパス '{train_dataset_path_raw}' が見つかりませんでした。")
        if not os.path.exists(validation_dataset_path_raw):
            raise FileNotFoundError(f"検証データセットのパス '{validation_dataset_path_raw}' が見つかりませんでした。")

        print(f"訓練データセットルートパス (変換前): {train_dataset_path_raw}")
        print(f"検証データセットルートパス (変換前): {validation_dataset_path_raw}")

        # --- ここから新しい変換処理 ---
        convert_to_pascal_voc_structure(train_dataset_path_raw)
        convert_to_pascal_voc_structure(validation_dataset_path_raw)

        # 変換後のパスをMediaPipe Model Makerに渡す
        train_dataset_path = train_dataset_path_raw # 変換後もルートパスは同じ
        validation_dataset_path = validation_dataset_path_raw # 変換後もルートパスは同じ

        print(f"訓練データセットルートパス (変換後): {train_dataset_path}")
        print(f"検証データセットルートパス (変換後): {validation_dataset_path}")

    else:
        raise FileNotFoundError("ZIPファイルがアップロードされませんでした。")

except ImportError:
    print("Google Colab以外の環境では、'files.upload()' は使用できません。")
    print("データセットのパスを手動で 'train_dataset_path' と 'validation_dataset_path' に設定してください。")
    # ここで手動でパスを設定する場合の例 (Google Colab以外の環境の場合)
    # train_dataset_path = "path/to/your/pascal_voc_dataset/train"
    # validation_dataset_path = "path/to/your/pascal_voc_dataset/test"
    raise SystemExit("ファイルのアップロード処理をスキップし、手動でのパス設定が必要です。")
except Exception as e:
    print(f"ファイル操作中にエラーが発生しました: {e}")
    raise


データセット(zipファイル)をアップロードしてください (PASCAL VOC形式, XML + JPEG)...


Saving business card.v1i.voc.zip to business card.v1i.voc.zip
'business card.v1i.voc.zip' を解凍しています...
解凍が完了しました。
訓練データセットルートパス (変換前): pascal_voc_dataset_extracted/train
検証データセットルートパス (変換前): pascal_voc_dataset_extracted/test
'pascal_voc_dataset_extracted/train' の構造を変換しています...
  XMLファイル 'photo_16_2024-04-01_16-23-01_jpg.rf.1f5e523ce0cdfe54b473ef6b6c287c7f.xml' から <path> タグを削除しました。
  XMLファイル 'photo_1_2024-04-01_16-21-05_jpg.rf.5f4e8bc94c15bc26169ed319e5d512fd.xml' から <path> タグを削除しました。
  XMLファイル 'photo_42_2024-04-01_16-21-05_jpg.rf.44f6dec0a224d0cc2de39b045ce35dcf.xml' から <path> タグを削除しました。
  XMLファイル 'photo_91_2024-04-01_16-21-05_jpg.rf.c2aa33926bbabc6850b41d8ea62e683e.xml' から <path> タグを削除しました。
  XMLファイル 'photo_20_2024-04-01_16-23-01_jpg.rf.d6c6c44337466d168385ac29886f7533.xml' から <path> タグを削除しました。
  XMLファイル 'photo_8_2024-04-01_16-23-01_jpg.rf.747f3827a6f0c3da0967d3bf7797e010.xml' から <path> タグを削除しました。
  XMLファイル 'photo_44_2024-04-01_16-21-05_jpg.rf.aa1fa57d6a0601f9b39ff4d8aaf35f34.xml' から <p

In [ ]:
# --- 2. データセットのロード ---
print("\nデータセットをロードしています...")
# train_dataとvalidation_dataをそれぞれ個別のパスからロードします。
try:
    train_data = object_detector.Dataset.from_pascal_voc_folder(
        train_dataset_path, cache_dir="/tmp/od_pascal_cache/train"
    )
    validation_data = object_detector.Dataset.from_pascal_voc_folder(
        validation_dataset_path, cache_dir="/tmp/od_pascal_cache/validation"
    )

    # テストデータは、検証データと同じものを使用するか、別途用意します
    test_data = validation_data

    print(f"訓練データセットサイズ: {train_data.size}")
    print(f"検証データセットサイズ: {validation_data.size}")
    print(f"テストデータセットサイズ: {test_data.size}")
    print(f"検出対象クラス: {train_data.label_names}")

except Exception as e:
    print(f"データセットのロード中にエラーが発生しました。PASCAL VOC形式のデータセット構造を確認してください: {e}")
    raise



データセットをロードしています...
訓練データセットサイズ: 381
検証データセットサイズ: 9
テストデータセットサイズ: 9
検出対象クラス: ['background', 'Company description', 'Company name', 'Job description', 'Phone number', 'Place', 'data-in-business-card', 'email', 'name', 'website']


In [ ]:

# --- 3. モデル訓練 ---
# MOBILENET_MULTI_AVG は、MediaPipe Solutionsで広く利用されている軽量なモデルアーキテクチャです。
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
#spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG_I384
#spec = object_detector.SupportedModels.EFFICIENTDET_LITE0


# ハイパーパラメータの設定
# エポック数とバッチサイズはデータセットの規模とGPUメモリに合わせて調整してください。
hparams = object_detector.HParams(
    export_dir='exported_pascal_voc_object_detector_model',
    epochs=50,  # 上限は50、ただし早期終了を使う
    batch_size=4,
    cosine_decay_epochs=40,  # 40エポック目から減衰開始
    cosine_decay_alpha=0.1   # 最終学習率の係数
)
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

print("\n物体検出モデルの訓練を開始します...")
try:
    model = object_detector.ObjectDetector.create(
        train_data=train_data,
        validation_data=validation_data,
        options=options
    )
    print("物体検出モデルの訓練が完了しました。")
except Exception as e:
    print(f"モデル訓練中にエラーが発生しました: {e}")
    print("データセットが正しく準備されているか、およびハイパーパラメータが適切か確認してください。")
    raise



物体検出モデルの訓練を開始します...


/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.11/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


95/95 [==============================] - 76s 371ms/step - total_loss: 3.6559 - cls_loss: 3.0295 - box_loss: 0.0113 - model_loss: 3.5930 - val_total_loss: 1.7776 - val_cls_loss: 1.2374 - val_box_loss: 0.0095 - val_model_loss: 1.7148
Epoch 2/50
95/95 [==============================] - 23s 238ms/step - total_loss: 1.6577 - cls_loss: 1.1604 - box_loss: 0.0087 - model_loss: 1.5949 - val_total_loss: 1.5930 - val_cls_loss: 1.0513 - val_box_loss: 0.0096 - val_model_loss: 1.5301
Epoch 3/50
95/95 [==============================] - 23s 239ms/step - total_loss: 1.4410 - cls_loss: 1.0096 - box_loss: 0.0074 - model_loss: 1.3782 - val_total_loss: 1.1915 - val_cls_loss: 0.8213 - val_box_loss: 0.0061 - val_model_loss: 1.1287
Epoch 4/50
95/95 [==============================] - 23s 240ms/step - total_loss: 1.2409 - cls_loss: 0.8479 - box_loss: 0.0066 - model_loss: 1.1781 - val_total_loss: 1.0793 - val_cls_loss: 0.6989 - val_box_loss: 0.0064 - val_model_loss: 1.0165
Epoch 5/50
95/95 [=====================

In [ ]:
# --- 4. モデル評価 ---
print("\nモデル評価を実行しています...")
try:
    # バッチサイズは訓練時と同じか、テストデータセットのサイズに合わせます。
    loss, coco_metrics = model.evaluate(test_data, batch_size=hparams.batch_size)

    # 変更点：lossがリストの場合、最初の要素を取り出す
    if isinstance(loss, list):
        # リストが空でなければ最初の要素を使用
        if len(loss) > 0:
            display_loss = loss[0]
        else:
            display_loss = float('nan') # リストが空の場合はNaNを表示
    else:
        display_loss = loss # リストでなければそのまま使用

    print(f"テストデータセットでの損失: {display_loss:.4f}")
    print(f"テストデータセットでのCOCOメトリクス: {coco_metrics}")
except Exception as e:
    print(f"モデル評価中にエラーが発生しました: {e}")
    raise




モデル評価を実行しています...
3/3 [==============================] - 1s 135ms/step - total_loss: 0.8515 - cls_loss: 0.5790 - box_loss: 0.0042 - model_loss: 0.7887
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.06s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.277
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.092
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.173
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.180
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.291
 Average Re

In [ ]:
# --- 5. モデルのエクスポートと量子化 ---
print("\nモデルのエクスポートを開始します...")

# float32 モデルのエクスポート
try:
    model.export_model('model_float32.tflite')
    print(f"Float32 モデルを '{hparams.export_dir}/model_float32.tflite' にエクスポートしました。")
except Exception as e:
    print(f"Float32モデルのエクスポート中にエラーが発生しました: {e}")

# int8 Quantization Aware Training (QAT) とエクスポート
print("\nQuantization Aware Training (QAT) を実行しています...")
try:
    # QATのエポック数は、訓練エポックとは別に設定します。
    # 通常、少量のエポックで十分なことが多いです。
    qat_hparams = object_detector.QATHparams(epochs=10)
    model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
    model.export_model('model_int8_qat.tflite')
    print(f"Int8 QAT モデルを '{hparams.export_dir}/model_int8_qat.tflite' にエクスポートしました。")
except Exception as e:
    print(f"Int8 QATモデルのエクスポート中にエラーが発生しました: {e}")

# float16 Post-Training Quantization (PTQ) とエクスポート
print("\nPost-Training Quantization (PTQ) (Float16) を実行しています...")
try:
    quantization_config_fp16 = quantization.QuantizationConfig.for_float16()
    model.restore_float_ckpt() # QAT実行後にfloatモデルの状態に戻してからPTQを適用
    model.export_model(model_name="model_fp16_ptq.tflite", quantization_config=quantization_config_fp16)
    print(f"Float16 PTQ モデルを '{hparams.export_dir}/model_fp16_ptq.tflite' にエクスポートしました。")
except Exception as e:
    print(f"Float16 PTQモデルのエクスポート中にエラーが発生しました: {e}")


print("\n--- エクスポートされたファイル ---")
# エクスポートされたファイルの一覧を表示
# os.systemを使用することで、Python環境でシェルコマンドを実行します。
os.system(f"ls -lh {hparams.export_dir}")

# Colab環境の場合、ファイルをダウンロードするためのヘルパー
try:
    print("\nモデルファイルをダウンロードできます:")
    files.download(os.path.join(hparams.export_dir, 'model_float32.tflite'))
    # 以下は、それぞれの量子化モデルが正常にエクスポートされた場合のみダウンロードを試みます
    if os.path.exists(os.path.join(hparams.export_dir, 'model_int8_qat.tflite')):
        files.download(os.path.join(hparams.export_dir, 'model_int8_qat.tflite'))
    if os.path.exists(os.path.join(hparams.export_dir, 'model_fp16_ptq.tflite')):
        files.download(os.path.join(hparams.export_dir, 'model_fp16_ptq.tflite'))
except Exception as e:
    print(f"ファイルのダウンロード中にエラーが発生しました: {e}")


# オプション: 作業用ディレクトリをクリーンアップ
# print("\n作業用ディレクトリをクリーンアップしています...")
# if os.path.exists(extract_dir):
#     shutil.rmtree(extract_dir)
# if os.path.exists("/tmp/od_pascal_cache"):
#     shutil.rmtree("/tmp/od_pascal_cache")
# print("クリーンアップが完了しました。")


モデルのエクスポートを開始します...
Exporting a floating point model


/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Float32 モデルを 'exported_pascal_voc_object_detector_model/model_float32.tflite' にエクスポートしました。

Quantization Aware Training (QAT) を実行しています...
Int8 QATモデルのエクスポート中にエラーが発生しました: module 'mediapipe_model_maker.python.vision.object_detector' has no attribute 'QATHparams'

Post-Training Quantization (PTQ) (Float16) を実行しています...


/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_1 (MobileNet)    {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn_1 (FPN)                 {'5': (None, 8, 8, 128)   144928    
                             , '4': (No

...

Float16 PTQ モデルを 'exported_pascal_voc_object_detector_model/model_fp16_ptq.tflite' にエクスポートしました。

--- エクスポートされたファイル ---

モデルファイルをダウンロードできます:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>